<a href="https://colab.research.google.com/github/nanfenggushi/colab-all-in-one-downloader/blob/main/colab_all_in_one_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===================================================================
# 步骤 1: 安装与验证 (高可靠性版)
# ===================================================================
print("🚀 开始安装所有必需的下载工具 (高可靠性模式)...")

print("\n--- 正在更新 apt 软件包列表 ---")
!apt-get update

print("\n--- 正在安装 aria2 ---")
!apt-get install -y aria2

print("\n--- 正在安装 Python 库 ---")
!pip -q install libtorrent yt-dlp requests

print("\n--- 强制安装后自检 ---")
import sys
from shutil import which

# 验证 aria2c
aria2c_path = which("aria2c")
if aria2c_path:
    print(f"✅ aria2c 安装成功！路径: {aria2c_path}")
    !aria2c --version
else:
    print("❌ 严重错误: aria2c 安装失败！无法在系统中找到 aria2c 命令。")
    print("💡 请检查上方 'apt-get install' 的输出日志，查找错误原因。")
    sys.exit("aria2c installation failed. Aborting.")

# 验证其他库
try:
    import libtorrent, requests
    print(f"✅ libtorrent, requests 安装成功！")
except ImportError as e:
    print(f"❌ 严重错误: Python 库导入失败: {e}")
    sys.exit("Python library installation failed. Aborting.")

print("\n✅ 所有工具已成功安装并验证！")


# ===================================================================
# 步骤 2: 连接Google Drive并准备所有核心下载功能
# ===================================================================
print("\n📚 正在导入库并准备核心功能...")
import os, time, threading
from urllib.parse import unquote
from google.colab import drive
import ipywidgets as widgets
from IPython.display import display, clear_output

print("🔗 正在请求连接到您的 Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("✅ Google Drive 连接成功！")
except Exception as e:
    print(f"❌ Google Drive 连接失败: {e}")

# --- 从上一步骤中获取 aria2c 的绝对路径 ---
ARIA2C_PATH = which("aria2c")

# --- 功能 1: 可续传流式下载 ---
def start_resumable_streaming_download(url, save_path, output_widget, manual_filename, daily_quota_gb=700):
    with output_widget:
        clear_output(wait=True)
        if not url.strip():
            print("⚠️ 请提供有效的下载链接。")
            return
        if manual_filename.strip():
            filename = manual_filename.strip()
            print(f"ℹ️ 使用手动指定的文件名: {filename}")
        else:
            try:
                filename = ""
                r_head = requests.head(url, timeout=10, allow_redirects=True)
                if 'content-disposition' in r_head.headers:
                    cd_header = r_head.headers['content-disposition']
                    filenames = [part.split('=')[-1].strip('"\'') for part in cd_header.split(';') if 'filename' in part.lower()]
                    if filenames:
                        filename = unquote(filenames[0])
                if not filename:
                    path = requests.utils.urlparse(r_head.url).path
                    filename = unquote(os.path.basename(path))
                if not filename:
                    raise ValueError("无法从URL自动获取文件名")
                print(f"ℹ️ 自动检测到的文件名: {filename}")
            except Exception as e:
                print(f"❌ 无法自动确定文件名: {e}。请手动指定文件名。")
                return

        destination = os.path.join(save_path, filename)
        os.makedirs(save_path, exist_ok=True)

        start_byte = 0
        file_mode = 'wb'
        headers = {}
        if os.path.exists(destination):
            start_byte = os.path.getsize(destination)
            print(f"检测到已存在的文件。将从字节 {start_byte} 处继续...")
            file_mode = 'ab'
            headers['Range'] = f'bytes={start_byte}-'
        else:
            print("开始新的下载任务...")

        daily_quota_bytes = daily_quota_gb * 1024**3
        print(f"每日配额设置为: {daily_quota_gb} GB\n目标文件: {destination}")

        try:
            global start_time
            start_time = time.time()
            with requests.get(url, stream=True, headers=headers, timeout=30) as r:
                if start_byte > 0 and r.status_code != 206:
                    print("❌ 错误: 服务器不支持断点续传。")
                    return
                r.raise_for_status()

                total_size = 0
                content_range = r.headers.get('content-range')
                if content_range:
                    total_size = int(content_range.split('/')[-1])
                else:
                    total_size = int(r.headers.get('content-length', 0)) + start_byte

                if total_size > 0 and total_size <= start_byte:
                    print("✅ 文件已存在且完整。")
                    return

                progress_bar = widgets.FloatProgress(value=start_byte, min=0, max=total_size if total_size > 0 else 1, description='下载中:', layout={'width': '50%'})
                progress_label = widgets.Label()
                display(widgets.HBox([progress_bar, progress_label]))
                downloaded_this_session = 0

                with open(destination, file_mode) as f:
                    for chunk in r.iter_content(chunk_size=256 * 1024**2):
                        if chunk:
                            f.write(chunk)
                            downloaded_this_session += len(chunk)
                            current_total_bytes = start_byte + downloaded_this_session
                            progress_bar.value = current_total_bytes
                            elapsed_time = time.time() - start_time
                            speed = downloaded_this_session / elapsed_time if elapsed_time > 0 else 0
                            progress_label.value = f"{current_total_bytes/1024**3:.2f}/{total_size/1024**3:.2f} GB (@{speed/1024**2:.2f}MB/s)"
                            if downloaded_this_session >= daily_quota_bytes:
                                print(f"\n✋ 已达到本次设定的 {daily_quota_gb} GB 下载配额！\n下载已自动暂停。请在24小时后再次运行此任务以继续。")
                                progress_bar.bar_style = 'warning'
                                progress_bar.description = '⏸️ 配额暂停'
                                return
                progress_bar.bar_style = 'success'
                progress_bar.description = '✅ 完成'
                print(f"\n🎉 文件 '{filename}' 已成功下载到您的Google Drive！")
        except requests.exceptions.RequestException as e:
            print(f"\n❌ 下载中断: {e}\n部分文件已保存。请稍后再次运行任务以继续。")
        except Exception as e:
            print(f"\n❌ 发生未知错误: {e}")

# --- 功能 2: 后台多线程HTTP下载 (语法修正) ---
def start_http_download(links, save_path, output_widget):
    if not ARIA2C_PATH:
        with output_widget:
            print("❌ 无法执行下载，因为没有找到 aria2c。")
        return

    with output_widget:
        clear_output(wait=True)
        os.makedirs(save_path, exist_ok=True)
        print(f"⬇️ 开始HTTP下载 (后台多线程)，保存到: {save_path}")

        link_list = [link.strip() for link in links.strip().split('\n') if link.strip()]
        if not link_list:
            print("⚠️ 链接列表为空。")
            return

        aria2_options = "-c -x 16 -s 16 -k 10M --max-concurrent-downloads=20 --summary-interval=10 --allow-overwrite=true"
        log_file = "/content/aria2_log.txt"

        command = f'nohup {ARIA2C_PATH} {aria2_options} -d "{save_path}" --log="{log_file}" --log-level=info'
        for link in link_list:
            command += f' "{link}"'
        command += f' >> "{log_file}" 2>&1 &'

        print("执行的命令:", command)
        get_ipython().system(command)

        print(f"\n🎉 {len(link_list)} 个HTTP下载任务已提交到后台。\nℹ️ 您可以关闭页面。")
        print(f"👀 请稍等几秒后，在新单元格中运行 !cat {log_file} 来查看详细日志和进度。")

# --- 功能 3: 后台BT下载 (语法修正) ---
def _bt_thread_worker(magnet_link, save_path, output_widget):
    def log(message):
        with output_widget:
            print(message)
    try:
        ses = lt.session({'user_agent': 'qBittorrent/4.4.2', 'listen_interfaces': '0.0.0.0:6881','connections_limit': 1000, 'download_rate_limit': 0, 'upload_rate_limit': 0,'cache_size': 2048, 'alert_mask': lt.alert.category_t.all_categories})
        params = lt.parse_magnet_uri(magnet_link)
        params.save_path = save_path
        handle = ses.add_torrent(params)

        log("正在获取种子元数据...")
        while not handle.has_metadata():
            time.sleep(1)

        s = handle.status()
        log(f"元数据获取成功！种子名称: {s.name}")

        while not s.is_seeding and s.state != lt.torrent_status.state_t.finished:
            s = handle.status()
            if int(time.time()) % 10 == 0:
                log(f"进度: {s.progress*100:.2f}% | 速度: DL {s.download_rate/1024**2:.2f}MB/s | Peers: {s.num_peers}")
            time.sleep(1)

        log(f"✅ 下载完成: {s.name}")
    except Exception as e:
        log(f"❌ BT下载线程发生严重错误: {e}")

def start_bt_download(magnet_link, save_path, output_widget):
    with output_widget:
        clear_output(wait=True)
        os.makedirs(save_path, exist_ok=True)
        print(f"🧲 准备启动BT下载 (后台模式)，保存到: {save_path}")

    if not magnet_link.strip().startswith("magnet:?"):
        with output_widget:
            print("⚠️ 无效的磁力链接。")
        return

    thread = threading.Thread(target=_bt_thread_worker, args=(magnet_link, save_path, output_widget))
    thread.daemon = True
    thread.start()

    with output_widget:
        print(f"\n🎉 BT任务已在后台线程中启动。\nℹ️ 您可以关闭页面。详细进度会在此处持续输出。")

# --- 功能 4: 后台YouTube及流媒体下载 (语法修正) ---
def start_youtube_download(url, save_path, output_widget):
    if not ARIA2C_PATH:
        with output_widget:
            print("❌ 无法执行下载，因为 yt-dlp 依赖 aria2c。")
        return

    with output_widget:
        clear_output(wait=True)
        os.makedirs(save_path, exist_ok=True)
        print(f"📹 开始YouTube/视频下载 (后台模式)，保存到: {save_path}")

        if not url.strip():
            print("⚠️ 请提供有效的视频链接。")
            return

        yt_dlp_options = f'--downloader aria2c --downloader-args "aria2c:{ARIA2C_PATH}" --concurrent-fragments 10'
        log_file = "/content/yt_dlp_log.txt"

        command = f'nohup yt-dlp {yt_dlp_options} -P "{save_path}" -f "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best" --merge-output-format mp4 --no-warnings "{url}" > "{log_file}" 2>&1 &'

        print("执行的命令:", command)
        get_ipython().system(command)

        print(f"\n🎉 视频下载任务已提交到后台。\nℹ️ 您可以关闭页面。")
        print(f"👀 查看进度: !tail -f {log_file}")

# ===================================================================
# 步骤 3: 启动下载器图形界面 (GUI)
# ===================================================================
print("\n✅ 所有核心功能已准备就绪！")
default_save_path = '/content/drive/MyDrive/Downloads'

# --- 界面 1: 可续传流式下载 ---
resumable_url_input = widgets.Text(placeholder='在此输入单个超大文件的HTTP/S直链', layout={'width': '98%'})
manual_filename_input = widgets.Text(value='', placeholder='(可选) 手动指定保存的文件名。留空则自动检测。', layout={'width': '98%'})
resumable_save_path_input = widgets.Text(value=default_save_path, description='保存路径:', layout={'width': '98%'})
daily_quota_input = widgets.BoundedFloatText(value=700, min=1, max=740, step=10, description='每日配额(GB):', style={'description_width': 'initial'})
resumable_button = widgets.Button(description="开始 / 继续下载", button_style='primary', icon='play', layout={'width': '98%'})
resumable_output = widgets.Output()
def on_resumable_button_clicked(b):
    start_resumable_streaming_download(resumable_url_input.value, resumable_save_path_input.value, resumable_output, manual_filename_input.value, daily_quota_input.value)
resumable_button.on_click(on_resumable_button_clicked)
resumable_tab = widgets.VBox([widgets.Label(value="【推荐】专为超大文件设计。自动断点续传和配额管理。下载期间需保持页面开启。"), resumable_save_path_input, resumable_url_input, manual_filename_input, daily_quota_input, resumable_button, resumable_output])

# --- 界面 2: 后台HTTP下载 ---
http_links_input = widgets.Textarea(placeholder='在此输入一个或多个HTTP/S直链，每行一个。', layout={'height': '100px', 'width': '98%'})
http_save_path_input = widgets.Text(value=default_save_path, description='保存路径:', layout={'width': '98%'})
http_button = widgets.Button(description="开始后台HTTP下载", button_style='success', icon='download', layout={'width': '98%'})
http_output = widgets.Output()
def on_http_button_clicked(b):
    start_http_download(http_links_input.value, http_save_path_input.value, http_output)
http_button.on_click(on_http_button_clicked)
http_tab = widgets.VBox([widgets.Label(value="使用aria2c后台多线程下载。速度快，可关闭页面。"), http_save_path_input, http_links_input, http_button, http_output])

# --- 界面 3: BT下载 ---
bt_magnet_input = widgets.Text(placeholder='在此输入一个磁力链接。', layout={'width': '98%'})
bt_save_path_input = widgets.Text(value=default_save_path, description='保存路径:', layout={'width': '98%'})
bt_button = widgets.Button(description="开始后台BT下载", button_style='info', icon='magnet', layout={'width': '98%'})
bt_output = widgets.Output()
def on_bt_button_clicked(b):
    start_bt_download(bt_magnet_input.value, bt_save_path_input.value, bt_output)
bt_button.on_click(on_bt_button_clicked)
bt_tab = widgets.VBox([widgets.Label(value="通过磁力链接后台下载文件。可关闭页面。"), bt_save_path_input, bt_magnet_input, bt_button, bt_output])

# --- 界面 4: YouTube下载 ---
yt_url_input = widgets.Text(placeholder='在此输入一个YouTube或其他支持的视频链接。', layout={'width': '98%'})
yt_save_path_input = widgets.Text(value=default_save_path, description='保存路径:', layout={'width': '98%'})
yt_button = widgets.Button(description="开始后台视频下载", button_style='danger', icon='youtube', layout={'width': '98%'})
yt_output = widgets.Output()
def on_yt_button_clicked(b):
    start_youtube_download(yt_url_input.value, yt_save_path_input.value, yt_output)
yt_button.on_click(on_yt_button_clicked)
yt_tab = widgets.VBox([widgets.Label(value="从YouTube等网站后台下载视频。可关闭页面。"), yt_save_path_input, yt_url_input, yt_button, yt_output])

# --- 创建并显示选项卡界面 ---
tab = widgets.Tab()
tab.children = [resumable_tab, http_tab, bt_tab, yt_tab]
tab.set_title(0, '🌊 可续传大文件')
tab.set_title(1, '🔗 后台HTTP')
tab.set_title(2, '🧲 BitTorrent')
tab.set_title(3, '📹 YouTube & More')

print("\n✅ 下载器界面已准备好，请在下方操作。")
display(tab)